# Notebook for visualizing and exploring the data

In [1]:
# Exploring the 'training' dataset
import pandas as pd
df = pd.read_csv("txt2sql_alerce_train_v2.csv")
df.head()

,req_id,request,table_info,external_knowledge,domain_knowledge,gold_query,difficulty,type,nested_type,rephrased_request,rephrased_request_gpt-3.5-turbo-0125_t0.4,rephrased_request_gpt-4o-2024-05-13_t0.2
0,13,Give me all the SNe that were first detected b...,"['object', 'probability']",\n-- mjd date for December = 59914.0\n-- mjd d...,\n-- Super Nova (SNe) is a large explosion tha...,"\nSELECT\n object.oid, probability.class_na...",simple,object,none,NaN,NaN,NaN
1,10,Get the object identifiers and probabilities i...,"['probability', 'object']",0,0,"\nSELECT\n sq1.oid, sq1.probability as SN_pro...",medium,object,simple,NaN,NaN,NaN
2,15,"Get the object identifiers, probabilities in t...","['object', 'probability', 'detection', 'magstat']",\n-- mjd date for September 01 = 60188.0\n-- m...,\n-- A fast riser is defined as an object whos...,"\nSELECT\n sq.oid, sq.probability, sq.candi...",advanced,other,multi,NaN,NaN,NaN
3,4,"Get the object identifier, candidate identifie...","['object', 'probability', 'magstat', 'detection']",\n-- mjd date for the start of the year 2019 =...,0,"\nSELECT\n sq.oid, sq.fid, sq.dmdt_first,\n ...",advanced,other,multi,NaN,NaN,NaN
4,25,Query objects within 10 degress of the next po...,"['probability', 'object']",\n-- mjd date for February 01 = 59976.0\n-- mj...,0,"\nWITH catalog ( source_id, ra, dec) AS (\n ...",advanced,spatial,simple,NaN,NaN,NaN


In [22]:
for request in df["request"]:
    if "SQL" in request:
        print(f"{df[df['request'] == request]['req_id'].values[0]}: {request}")

32: Given a list of oids of asteroids candidates, write a SQL script that returns data from the ss_ztf table with a positive ssdistnr value.
20: Query objects classified by the light curve classifier that are most likely to be RRL with a probability larger than 0.55 and that have 100 or more detections. Write in PostgresSQL.
18: Write a script in PostgreSQL with a query asking for all the probabilities for all the sources classfied as either AGN, QSO or Blazar by the light curve classifier. Define a minimum number of detections of 40, and a minimum time span between the first and the last detection of 200 days, in order to ensure the detection of a variable signal from the light curves. Also, filter out sources using a predicted class probability higher than 0.8 for the Blazar class and 0.95 for the AGN and QSO classes. Collect similar numbers for each class using a UNION statement between each sub-query defined by each probability. The query must return the columns oid, meanra, meande

In [7]:
print(df[df["difficulty"] == "advanced"]["request"][2])
print(df[df["difficulty"] == "advanced"]["gold_query"][2])

Get the object identifiers, probabilities in the stamp classifier and light curves (only detections) for objects whose highest probability in the stamp classifier is obtained for class SN, that had their first detection in the first 2 days of september, and that qualify as fast risers.

SELECT
    sq.oid, sq.probability, sq.candid, sq.fid, sq.mjd,
    magstat.fid as magstat_fid, magstat.dmdt_first
FROM
  (
SELECT
  detection.oid, detection.candid, detection.fid, detection.mjd,
  obj_oids.probability
FROM
  (
SELECT
    object.oid, probability.probability
FROM
    object INNER JOIN
    probability
    ON object.oid = probability.oid
WHERE
    probability.classifier_name='stamp_classifier'
    AND probability.class_name='SN'
    AND probability.ranking=1
    AND object.firstmjd > 60188.0
    AND object.firstmjd < 60189.0
) as obj_oids
    INNER JOIN
    detection ON detection.oid = obj_oids.oid
) AS sq
  INNER JOIN magstat
  ON sq.oid = magstat.oid
WHERE
  magstat.dmdt_first < -0.25
ORDE

In [8]:
# Step 2: Retrieve objects classified as 'SN' by the stamp classifier
sub_query_probabilities = """
SELECT oid, probability 
FROM probability 
WHERE class_name = 'SN' 
AND classifier_name = 'stamp_classifier' 
AND ranking = 1
"""

# Step 3: Filter objects with first detection in the first two days of September
sub_query_first_detection = """
SELECT oid 
FROM detection 
WHERE (timestamp '1970-01-01' + (mjd + 2400000.5 - 2440587.5) * INTERVAL '1 day')::date 
BETWEEN '2024-09-01' AND '2024-09-02'
AND isdiffpos = 1
"""

# Step 4: Identify objects qualifying as fast risers based on features
sub_query_fast_risers = """
SELECT oid 
FROM feature 
WHERE (name = 'dmdt_first' AND value > 0.5)  -- Example of fast rising criteria, adjust threshold as needed
AND (name = 'dm_first' AND value > 0.3)
AND (name = 'delta_mjd_fid' AND value < 2)
"""

# Step 5: Retrieve light curves (detections) for filtered objects
sub_query_light_curves = """
SELECT oid, mjd, magpsf 
FROM detection 
WHERE isdiffpos = 1
"""

# Step 6: Combine the results from subqueries
full_query = f"""
SELECT p.oid, p.probability, d.mjd, d.magpsf
FROM ({sub_query_probabilities}) p
INNER JOIN ({sub_query_first_detection}) fd ON p.oid = fd.oid
INNER JOIN ({sub_query_fast_risers}) fr ON p.oid = fr.oid
INNER JOIN ({sub_query_light_curves}) d ON p.oid = d.oid
"""

In [9]:
from secret.config import SQL_URL
import requests
import sqlalchemy as sa

# Setup params for query engine
params = requests.get(SQL_URL).json()['params']
engine = sa.create_engine(f"postgresql+psycopg2://{params['user']}:{params['password']}@{params['host']}/{params['dbname']}")
engine.begin()

pd.read_sql_query(full_query, con=engine)

,oid,probability,mjd,magpsf
